# Explainability with LLM-Generated Narratives

This notebook generates human-readable explanations for top fraud detections using attention weights and LLM.

## Overview
1. Load quantum model with attention
2. Generate predictions with attention weights
3. Extract explanations for top fraud cases
4. Use LLM to generate forensic narratives
5. Save explanations to JSON

**Estimated time:** 2-5 minutes (+ LLM API calls)

**Note:** Requires OpenAI API key in `.env` file for LLM explanations.

In [ ]:
# Clear any cached src imports
import sys
if 'src' in sys.modules:
    del sys.modules['src']
for key in list(sys.modules.keys()):
    if key.startswith('src.'):
        del sys.modules[key]
print("✓ Cleared cached imports")

In [ ]:
# Cell 1: Import libraries and setup
import sys
import torch
import torch.nn.functional as F
import numpy as np
import os
import json

# Try to load environment variables if .env exists
try:
    from dotenv import load_dotenv
    load_dotenv()
except ImportError:
    print("⚠️ python-dotenv not installed. Skipping .env loading.")
except Exception as e:
    print(f"⚠️ Could not load .env: {e}")

# Always add project root to sys.path for src imports (Jupyter-safe)
notebook_dir = os.path.abspath('')
project_root = os.path.dirname(notebook_dir)
if project_root not in sys.path:
    sys.path.insert(0, project_root)
    print(f"Added project root to sys.path: {project_root}")

from src.models import GAT
from src.config import MODEL_CONFIG, EXPLAIN_CONFIG, ARTIFACTS_DIR, ARTIFACT_FILES, FIGURES_DIR, FIGURE_FILES
from src.utils import get_device

device = get_device()
print(f"Using device: {device}")

Device: cpu
Using device: cpu


In [ ]:
# Cell 2: Load quantum model and data
from sklearn.model_selection import train_test_split

graph_path = ARTIFACTS_DIR / ARTIFACT_FILES['quantum_graph']
data = torch.load(graph_path, map_location=device, weights_only=False)

# Create test mask if it doesn't exist (needed for evaluation)
if not hasattr(data, 'test_mask'):
    print("Creating train/val/test splits...")
    labeled_indices = torch.where(data.labeled_mask)[0].cpu().numpy()
    labeled_y = data.y[data.labeled_mask].cpu().numpy()
    
    train_val_idx, test_idx = train_test_split(
        labeled_indices, test_size=0.2, 
        random_state=MODEL_CONFIG.get('random_seed', 42),
        stratify=labeled_y
    )
    
    test_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
    test_mask[test_idx] = True
    data.test_mask = test_mask.to(device)
    print(f"  Test set size: {test_mask.sum()}")

model = GAT(
    in_channels=data.num_node_features,
    hidden_channels=MODEL_CONFIG['hidden_channels'],
    out_channels=MODEL_CONFIG['out_channels'],
    num_heads=MODEL_CONFIG['num_heads'],
    num_layers=MODEL_CONFIG['num_layers'],
    dropout=MODEL_CONFIG['dropout']
).to(device)

model_path = ARTIFACTS_DIR / ARTIFACT_FILES['quantum_model']
checkpoint = torch.load(model_path, map_location=device, weights_only=False)

# Handle different checkpoint formats
if isinstance(checkpoint, dict) and 'model_state_dict' in checkpoint:
    model.load_state_dict(checkpoint['model_state_dict'])
else:
    model.load_state_dict(checkpoint)
    
model.eval()

print(f"✓ Model loaded with {data.num_node_features} features")

Creating train/val/test splits...
  Test set size: 9313
✓ Model loaded with 364 features


In [ ]:
# Cell 3: Generate predictions with attention weights
@torch.no_grad()
def get_predictions_with_attention():
    out, attention_weights = model(data.x, data.edge_index, return_attention_weights=True)
    probs = F.softmax(out, dim=1)
    return probs, attention_weights

probabilities, attention_weights = get_predictions_with_attention()
print(f"✓ Generated predictions with attention weights")
print(f"  Number of attention layers: {len(attention_weights)}")

✓ Generated predictions with attention weights
  Number of attention layers: 2


In [ ]:
# Cell 4: Load top fraud cases
fraud_indices_path = ARTIFACTS_DIR / ARTIFACT_FILES['top_fraud_indices']
top_fraud_idx = np.load(fraud_indices_path)
print(f"✓ Loaded top {len(top_fraud_idx)} fraud cases")
print(f"  Fraud indices: {top_fraud_idx[:5]}...")

✓ Loaded top 10 fraud cases
  Fraud indices: [140118 117413 115106  74470 117434]...


In [ ]:
# Cell 5: Define explanation extraction function
def explain_node(node_idx, k_neighbors=None):
    """Generate explanation for a specific node."""
    
    if k_neighbors is None:
        k_neighbors = EXPLAIN_CONFIG['k_neighbors']
    
    # Get fraud probability
    fraud_prob = probabilities[node_idx, 1].item()
    
    # Get attention weights from last layer
    edge_index, attn = attention_weights[-1]
    edge_index = edge_index.cpu().numpy()
    attn = attn.squeeze().cpu().numpy()
    
    # Find edges connected to this node
    incoming_mask = edge_index[1] == node_idx
    incoming_edges = edge_index[0][incoming_mask]
    incoming_attn = attn[incoming_mask]
    
    # Get top k neighbors by attention
    if len(incoming_edges) > 0:
        top_k = min(k_neighbors, len(incoming_edges))
        top_indices = np.argsort(incoming_attn)[-top_k:][::-1]
        top_neighbors = incoming_edges[top_indices]
        top_attention = incoming_attn[top_indices]
    else:
        top_neighbors = np.array([])
        top_attention = np.array([])
    
    # Get node features (first 10 for brevity)
    node_features = data.x[node_idx].cpu().numpy()[:10]
    
    # Get neighbor labels
    neighbor_labels = []
    for neighbor in top_neighbors:
        label = data.y[neighbor].item()
        if label == 1:
            neighbor_labels.append("illicit")
        elif label == 0:
            neighbor_labels.append("licit")
        else:
            neighbor_labels.append("unknown")
    
    return {
        'node_idx': int(node_idx),
        'fraud_probability': fraud_prob,
        'true_label': int(data.y[node_idx].item()),
        'timestep': int(data.timestep[node_idx].item()),
        'top_features': node_features.tolist(),
        'top_neighbors': top_neighbors.tolist(),
        'neighbor_attention': top_attention.tolist(),
        'neighbor_labels': neighbor_labels
    }

# Test explanation
explanation = explain_node(top_fraud_idx[0])
print(f"\n✓ Example explanation for node {explanation['node_idx']}:")
print(f"  Fraud probability: {explanation['fraud_probability']:.4f}")
print(f"  True label: {'illicit' if explanation['true_label'] == 1 else 'licit'}")
print(f"  Timestep: {explanation['timestep']}")
print(f"  Top {len(explanation['neighbor_labels'])} neighbor labels: {explanation['neighbor_labels']}")


✓ Example explanation for node 140118:
  Fraud probability: 0.5089
  True label: licit
  Timestep: 35
  Top 5 neighbor labels: ['illicit', 'illicit', 'illicit', 'illicit', 'illicit']


In [ ]:
# Cell 6: Define LLM explanation generator
def generate_llm_explanation(explanation):
    """Use LLM to generate human-readable explanation."""
    
    # Check if LLM is enabled
    if not EXPLAIN_CONFIG['use_llm']:
        return "LLM explanation disabled in config."
    
    # Check if OpenAI API key is available
    api_key = os.getenv('OPENAI_API_KEY')
    if not api_key:
        return "⚠️ OpenAI API key not found. Set OPENAI_API_KEY in .env file or environment."
    
    try:
        from openai import OpenAI
        client = OpenAI(api_key=api_key)
        
        # Build prompt
        true_label_str = "illicit" if explanation['true_label'] == 1 else "licit" if explanation['true_label'] == 0 else "unknown"
        
        prompt = f"""You are a fraud detection analyst. Generate a concise forensic narrative explaining why this Bitcoin transaction was flagged as potentially fraudulent.

Transaction Details:
- Node ID: {explanation['node_idx']}
- Fraud Probability: {explanation['fraud_probability']:.2%}
- True Label: {true_label_str}
- Timestep: {explanation['timestep']}
- Top Feature Values: {', '.join([f'{v:.3f}' for v in explanation['top_features'][:5]])}
- Connected to {len(explanation['top_neighbors'])} key neighbors
- Neighbor Labels: {', '.join(explanation['neighbor_labels'])}
- Attention Weights: {', '.join([f'{a:.3f}' for a in explanation['neighbor_attention']])}

Generate a 3-4 sentence explanation for investigators, focusing on:
1. Why this transaction is suspicious
2. What patterns the model detected
3. Key connections to other transactions

Be specific and technical but readable."""

        response = client.chat.completions.create(
            model=EXPLAIN_CONFIG['llm_model'],
            messages=[{"role": "user", "content": prompt}],
            temperature=0.7,
            max_tokens=200
        )
        
        return response.choices[0].message.content
        
    except ImportError:
        return "⚠️ OpenAI library not installed. Run: pip install openai"
    except Exception as e:
        return f"⚠️ LLM explanation failed: {str(e)}"

print("✓ LLM explanation function defined")
print(f"  LLM enabled: {EXPLAIN_CONFIG['use_llm']}")
print(f"  Model: {EXPLAIN_CONFIG['llm_model']}")
print(f"  API key available: {'Yes' if os.getenv('OPENAI_API_KEY') else 'No'}")

✓ LLM explanation function defined
  LLM enabled: True
  Model: gpt-4o-mini
  API key available: No


In [ ]:
# Cell 7: Generate explanations for top fraud cases
print("\n" + "="*80)
print("FRAUD DETECTION EXPLANATIONS")
print("="*80)

explanations_output = []
num_explanations = min(EXPLAIN_CONFIG['num_explanations'], len(top_fraud_idx))

for i, node_idx in enumerate(top_fraud_idx[:num_explanations], 1):
    print(f"\n{'='*80}")
    print(f"CASE #{i} - Node {node_idx}")
    print('='*80)
    
    # Get explanation data
    exp = explain_node(node_idx)
    
    # Basic info
    true_label_str = "ILLICIT" if exp['true_label'] == 1 else "LICIT" if exp['true_label'] == 0 else "UNKNOWN"
    print(f"\n📊 Fraud Score: {exp['fraud_probability']:.2%}")
    print(f"🏷️  True Label: {true_label_str}")
    print(f"⏰ Timestep: {exp['timestep']}")
    
    if len(exp['top_neighbors']) > 0:
        print(f"\n🔗 Top {len(exp['top_neighbors'])} Connected Nodes (by attention):")
        for j, (neighbor, attn, label) in enumerate(zip(exp['top_neighbors'], 
                                                          exp['neighbor_attention'], 
                                                          exp['neighbor_labels']), 1):
            print(f"  {j}. Node {neighbor:6d} - Attention: {attn:.4f} - Label: {label}")
    else:
        print("\n🔗 No connected nodes found")
    
    # Generate LLM explanation
    print(f"\n📝 Forensic Narrative:")
    llm_explanation = generate_llm_explanation(exp)
    print(llm_explanation)
    
    explanations_output.append({
        'case_number': i,
        'node_idx': exp['node_idx'],
        'fraud_score': exp['fraud_probability'],
        'true_label': true_label_str,
        'timestep': exp['timestep'],
        'num_neighbors': len(exp['top_neighbors']),
        'narrative': llm_explanation
    })

print(f"\n{'='*80}")
print(f"✓ Generated {len(explanations_output)} explanations")
print('='*80)


FRAUD DETECTION EXPLANATIONS

CASE #1 - Node 140118

📊 Fraud Score: 50.89%
🏷️  True Label: UNKNOWN
⏰ Timestep: 35

🔗 Top 5 Connected Nodes (by attention):
  1. Node 139451 - Attention: 0.0433 - Label: illicit
  2. Node 139452 - Attention: 0.0433 - Label: illicit
  3. Node 139483 - Attention: 0.0429 - Label: illicit
  4. Node 139464 - Attention: 0.0427 - Label: illicit
  5. Node 139472 - Attention: 0.0426 - Label: illicit

📝 Forensic Narrative:
⚠️ OpenAI API key not found. Set OPENAI_API_KEY in .env file or environment.

CASE #2 - Node 117413

📊 Fraud Score: 50.91%
🏷️  True Label: UNKNOWN
⏰ Timestep: 29

🔗 Top 5 Connected Nodes (by attention):
  1. Node 117446 - Attention: 0.0160 - Label: unknown
  2. Node 117443 - Attention: 0.0158 - Label: unknown
  3. Node 117468 - Attention: 0.0158 - Label: unknown
  4. Node 117413 - Attention: 0.0158 - Label: unknown
  5. Node 117457 - Attention: 0.0157 - Label: unknown

📝 Forensic Narrative:
⚠️ OpenAI API key not found. Set OPENAI_API_KEY in .env

In [ ]:
# Cell 8: Save explanations to JSON
save_path = ARTIFACTS_DIR / ARTIFACT_FILES['fraud_explanations']
with open(save_path, 'w') as f:
    json.dump(explanations_output, f, indent=2)

print(f"\n✓ Explanations saved to {save_path}")
print(f"  Total cases explained: {len(explanations_output)}")


✓ Explanations saved to c:\Users\tusha\Documents\UT_Dallas\ACM_SP26\imple2\artifacts\fraud_explanations.json
  Total cases explained: 10


In [ ]:
# Cell 9: Project completion summary
print("\n" + "="*80)
print("🎉 AEGIS PROJECT COMPLETE")
print("="*80)
print("\n✅ All notebooks completed successfully!")
print("\n📦 Generated Artifacts:")
print(f"  • {ARTIFACT_FILES['baseline_graph']}")
print(f"  • {ARTIFACT_FILES['quantum_graph']}")
print(f"  • {ARTIFACT_FILES['baseline_model']}")
print(f"  • {ARTIFACT_FILES['quantum_model']}")
print(f"  • {ARTIFACT_FILES['baseline_metrics']}")
print(f"  • {ARTIFACT_FILES['quantum_metrics']}")
print(f"  • {ARTIFACT_FILES['top_fraud_indices']}")
print(f"  • {ARTIFACT_FILES['fraud_explanations']}")

print("\n📊 Generated Figures:")
print(f"  • {FIGURE_FILES['data_distribution']}")
print(f"  • {FIGURE_FILES['baseline_confusion']}")
print(f"  • {FIGURE_FILES['baseline_roc']}")
print(f"  • {FIGURE_FILES['quantum_features']}")
print(f"  • {FIGURE_FILES['quantum_confusion']}")
print(f"  • {FIGURE_FILES['comparison']}")
print(f"  • {FIGURE_FILES['roc_comparison']}")

print("\n" + "="*80)
print("Your quantum-inspired, explainable fraud detection system is ready!")
print("="*80)


🎉 AEGIS PROJECT COMPLETE

✅ All notebooks completed successfully!

📦 Generated Artifacts:
  • elliptic_graph.pt
  • elliptic_graph_quantum.pt
  • gat_baseline.pt
  • gat_quantum.pt
  • gat_baseline_metrics.json
  • gat_quantum_metrics.json
  • top_fraud_indices.npy
  • fraud_explanations.json

📊 Generated Figures:
  • data_distribution.png
  • baseline_confusion_matrix.png
  • baseline_roc_curve.png
  • quantum_feature_distribution.png
  • quantum_confusion_matrix.png
  • baseline_vs_quantum_comparison.png
  • roc_comparison.png

Your quantum-inspired, explainable fraud detection system is ready!


---

## 🎉 AEGIS PROJECT COMPLETE!

All notebooks have been executed successfully. Your quantum-inspired, explainable fraud detection system is ready for analysis and presentation!